In [ ]:
import numpy as np
from ipywidgets import *
import matplotlib.pyplot as plt
import datetime
from mpl_toolkits.basemap import Basemap, shiftgrid
import pygrib
import glob
import string
from shapely.geometry import Point, mapping
from fiona import collection
from IPython.html import widgets
import copy
import urllib
import netCDF4
import scipy.interpolate

%matplotlib notebook

In [ ]:
SAVEPATH='/mnt/pandora/AegeanSea/'

Read all forecast for wind and pinpoint maximum wind time frame.

In [ ]:
from gribapi import *
from redtoreg import _redtoreg
from pygrib import gaulats

def gridd(lon1,lat1,lon2,lat2,nlats):

        #   lon1, lat1 = self.longitude_first_gridpoint, self.latitude_first_gridpoint
        #   lon2, lat2 = self.longitude_last_gridpoint, self.latitude_last_gridpoint
        #   nlats = self.points_in_y_direction
            # ECMWF 'reduced' gaussian grid.
            nlons = 2*nlats
            delon = 360./nlons
        #   lons = np.arange(lon1,lon2,delon)
            lons = np.linspace(lon1,lon2,nlons)
            # compute gaussian lats (north to south)
            lats = gaulats(nlats)
            if lat1 > lat2 :
               lats = lats[::-1]
          # lons = lons[::-1]
            lons,lats = np.meshgrid(lons,lats) # make 2-d arrays

            return lons,lats


def reade(INPUT,time):
    f = open(INPUT)

    for t in range(3*time):
        gid = grib_new_from_file(f,headers_only = True)
    dat={}
    for l in range(3):
        gid = grib_new_from_file(f)
        if gid is None: break

        name=grib_get(gid, 'shortName')
        ss=grib_get_array(gid,'pl')  # lons per lat for the reduced_gg grid
        mv=grib_get(gid,'missingValue')

        lonfgp=grib_get(gid,'longitudeOfFirstGridPointInDegrees')
        latfgp=grib_get(gid,'latitudeOfFirstGridPointInDegrees')
        lonlgp=grib_get(gid,'longitudeOfLastGridPointInDegrees')
        latlgp=grib_get(gid,'latitudeOfLastGridPointInDegrees')

        lon,lat = gridd(lonfgp,latfgp,lonlgp,latlgp,ss.size)

        dat[name]=grib_get_values(gid)

        grib_release(gid)

    f.close()

    return mv,ss,dat,lat,lon

def  getd(INPUT,time):


        [mv,ss,dat,lat,lon]=reade(INPUT,time)

        ny=2*np.size(ss)

        for key,values in dat.iteritems():
                dat[key]=_redtoreg(ny,ss,values,mv)

        return dat,lon,lat


function to save to geotiff file 

In [ ]:
from osgeo import gdal,gdal_array
import osr

dataTypeformat={1:np.byte,2:np.int32,3:np.int32,4:np.float32,5:np.float32,6:np.byte}
VSType={1:'VS_BOOLEAN',2:'VS_NOMINAL',3:'VS_ORDINAL',4:'VS_SCALAR',5:'VS_DIRECTION',6:'VS_LDD'}

def putmap(filename,var,geo,TYPE,nodata):
     driver=gdal.GetDriverByName('PCRaster')
     varw=var.astype(dataTypeformat[TYPE])
     gtype=gdal_array.NumericTypeCodeToGDALTypeCode(varw.dtype)
     NROWS,NCOLS = var.shape
     VS='PCRASTER_VALUESCALE={}'.format(VSType[TYPE])
     dst_ds=driver.Create(filename,NCOLS,NROWS,1,gtype,[VS])
     proj=osr.SpatialReference()
     proj.ImportFromEPSG(4326)
     dst_ds.SetProjection(proj.ExportToWkt())
     dst_ds.SetGeoTransform(geo)
     dst_ds.GetRasterBand(1).WriteArray(varw)
     dst_ds.GetRasterBand(1).SetNoDataValue(nodata)
     dst_ds.FlushCache()
     dst_ds=None
     return


We have to create a mask for the area we want. We also include the bathymetry in order to get the max wind over sea.

In [ ]:
# set PATH of the database.
PATHbase="/mnt/ECMWF/grib/"  # Local mapping location for the above network drive


In [ ]:
def CreateMask(minlon=5.,maxlon=48.,minlat=28.,maxlat=45.):
    yday=datetime.date.today()-datetime.timedelta(days=1)
    
    PATH=PATHbase+"%04i/%02i/%02i/" % (yday.year,yday.month,yday.day)

    dpath=glob.glob(PATH+"*%04i%02i%02i.%02i.tropical_cyclone.grib" % (yday.year,yday.month,yday.day,0))

    try:
         data,lons,lats = getd(dpath[0],0) 
    except:        
         print 'no available data in ', PATH
         return
      
    w=(lats > minlat) & (lats < maxlat) # mask latitudes
    z=(lons > minlon) & (lons < maxlon) # mask longitudes BE CAREFUL OF & AND | DEPENDING
    d=w&z # joined mask

    # get topography
    # Read data from: http://coastwatch.pfeg.noaa.gov/erddap/griddap/usgsCeSrtm30v6.html
    # using the netCDF output option
    base_url='http://coastwatch.pfeg.noaa.gov/erddap/griddap/usgsCeSrtm30v6.nc?'
    query='topo[(%f):%d:(%f)][(%f):%d:(%f)]' % (maxlat,5,minlat,minlon,5,maxlon)
    url = base_url+query
    # store data in NetCDF file?
    file='usgsCeSrtm30v6.nc'
    urllib.urlretrieve (url, file)

   # open NetCDF data in
    nc = netCDF4.Dataset(file)
    ncv = nc.variables
    #print ncv.keys()

    lon = ncv['longitude'][:]
    lat = ncv['latitude'][:]
    topo = ncv['topo'][:,:]
    # interpolate on the wind grid
    # flip lat in order to make it increasing
    lat=np.flipud(lat)
    # evaluate the interpolating function 
    f=scipy.interpolate.interp2d(lon,lat,np.flipud(topo),kind='cubic') # note that we also flip topo due to the flip of lat above
    
    #replicate the wind grid
    alat=lats[:,0]
    alon=lons[0,:]
    # mask the 1-D arrays
    aw=(alat > minlat) & (alat < maxlat)
    az=(alon > minlon) & (alon < maxlon)
    wlat=alat[aw]
    wlon=alon[az]
    # interpolate on the new grid for the topography on the wind grid
    itopo = f(wlon,wlat)
    
    # mask positive values   
    b=itopo<0.
    # create a mask of wind shape 
    mask=np.zeros(lats.shape,dtype=bool)
    # modify the window mask to account only for the sea points
    mask[d]=b.flatten()
    #total mask on the window and over sea
    
    return mask
    

Define the window of interest

In [ ]:
minlat=30.
maxlat=41.5
minlon=18.
maxlon=36.


In [ ]:
mask=CreateMask(minlon,maxlon,minlat,maxlat)

Scan the forecast for the maximum wind

In [ ]:
def maxv(yyyy=2016,mm=1,dd=25,hh=0,save=False):
    hh=np.int(hh) # The variables are passed as string and needs to become integer to be used later
    yyyy=np.int(yyyy)
    mm=np.int(mm)
    dd=np.int(dd)

    # specify date to plot.
    date = datetime.datetime(yyyy,mm,dd,hh)

    # set PATH of the database.
    PATHbase="/mnt/Tsunamiweb/grib/"  # Local mapping location for the above network drive
  # PATHbase="/mnt/ECMWF/grib/"  # Local mapping location for the above network drive
    PATH=PATHbase+"%04i/%02i/%02i/" % (yyyy,mm,dd)
    
    dpath=glob.glob(PATH+"*%04i%02i%02i.%02i.tropical_cyclone.grib" % (yyyy,mm,dd,hh))

    maxvel=0.0
    for time in range(0,49,3):

       try:
         data,lons,lats = getd(dpath[0],time) 
       except:        
         print 'no available data in ', PATH
         return
    # read u,v
       ud=np.flipud(data['10u']) # because of the flip of lats
       vd=np.flipud(data['10v']) # because of the flip of lats
       u=np.ma.masked_array(ud,mask=np.invert(mask)) # invert mask for setting true the window values
       v=np.ma.masked_array(vd,mask=np.invert(mask)) # invert mask for setting true the window values
    # 
       vel=np.sqrt(u**2+v**2)
       if vel.max() > maxvel:
           maxvel=max([maxvel,vel.max()])
           timestamp=time

    return maxvel, timestamp

In [ ]:
#test
#val,t=maxv(2016,1,25,0)

The plot the graph for the timestamp..

The interactive function is below

In [ ]:
def wmap(yyyy=2016,mm=1,dd=28,hh=12,save=False):
    hh=np.int(hh) # The variables are passed as string and needs to become integer to be used later
    yyyy=np.int(yyyy)
    mm=np.int(mm)
    dd=np.int(dd)
    # specify date to plot.
    date = datetime.datetime(yyyy,mm,dd,hh)

    # set PATH of the database.
    PATHbase="/mnt/Tsunamiweb/grib/"  # Local mapping location for the above network drive
  # PATHbase="/mnt/ECMWF/grib/"  # Local mapping location for the above network drive
    PATH=PATHbase+"%04i/%02i/%02i/" % (yyyy,mm,dd)

    dpath=glob.glob(PATH+"*%04i%02i%02i.%02i.tropical_cyclone.grib" % (yyyy,mm,dd,hh))

    # get the timestamp of the maximum air speed
    val,tmax=maxv(yyyy,mm,dd,hh)
    
    # plot the map of this timestamp
    
    try:
        data,lon,lat = getd(dpath[0],tmax) # for faster reading THERE ARE SOME ISSUES WITH IT
    except:
        print 'no available data in ', PATH
        return

    # get sea level pressure and 10-m wind data.
  #  pd=data['msl'] # because of the flip of lats
    ud=data['10u'] # here we should flip because of the flip of lats but it is done below manually
    vd=data['10v'] # here we should flip because of the flip of lats but it is done below manually
    
    # read lats,lons
    latitudes = lat[:,0]
    longitudes = lon[0,:]
    # mult slp by 0.01 to put in units of hPa
   # slpin = 0.01*pd.squeeze()
    uin=ud.squeeze()
    vin=vd.squeeze()


    # add cyclic points manually (could use addcyclic function) NOTE THAT WE ALSO FLIP.. SEE ABOVE
#    slp= np.zeros((slpin.shape[0],slpin.shape[1]+1),np.float64)
#    slp[:,0:-1] = slpin[::-1]; slp[:,-1] = slpin[::-1,0]
    u= np.zeros((uin.shape[0],uin.shape[1]+1),np.float64)
    u[:,0:-1] = uin[::-1]; u[:,-1] = uin[::-1,0]
    v= np.zeros((vin.shape[0],vin.shape[1]+1),np.float64)
    v[:,0:-1] = vin[::-1]; v[:,-1] = vin[::-1,0]

    longitudes=np.append(longitudes,360.)

    lons, lats = np.meshgrid(longitudes,latitudes)

    # make orthographic basemap.
    m = Basemap(projection='cyl',llcrnrlat=minlat,urcrnrlat=maxlat,\
             llcrnrlon=minlon,urcrnrlon=maxlon,resolution='c')
    # create figure, add axes
    fig1 = plt.figure(figsize=(8,10))
    ax = fig1.add_axes([0.1,0.1,0.8,0.8])
    # set desired contour levels.
    clevs = np.arange(960,1061,5)

    # compute native x,y coordinates of grid.
    x, y = m(lons, lats)

    # define parallels and meridians to draw.
    parallels = np.arange(-90.,90,20.)
    meridians = np.arange(0.,360.,20.)
    # plot SLP contours.
 #   slpg,newlons = shiftgrid(180.,slp,longitudes,start=False)
 #   slpd,xx,yy = \
 #   m.transform_scalar(slpg,newlons,latitudes,181,181,returnxy=True,masked=True)
 #   CS1 = m.contour(x,y,slp,clevs,linewidths=0.5,colors='k',animated=True)
 #   CS2 = m.contourf(x,y,slp,clevs,cmap=plt.cm.RdBu_r,animated=True)
    # plot wind vectors on projection grid.
    # first, shift grid so it goes from -180 to 180 (instead of 0 to 360
    # in longitude).  Otherwise, interpolation is messed up.
    ugrid,newlons = shiftgrid(180.,u,longitudes,start=False)
    vgrid,newlons = shiftgrid(180.,v,longitudes,start=False)
    
    nn=81
    dd=(maxlon-minlon)/nn
    nj=np.int((maxlat-minlat)/dd)
    ni=np.int((maxlon-minlon)/dd)
   # print ni,nj
    # transform vectors to projection grid.
    uproj,vproj,xx,yy = \
    m.transform_vector(ugrid,vgrid,newlons,latitudes,ni,nj,returnxy=True,masked=True)
    # now plot.
    vel=np.sqrt(uproj**2+vproj**2)
    
    CS1 = m.contour(xx,yy,vel,10,linewidths=0.5,colors='k',animated=True)
    CS2 = m.contourf(xx,yy,vel,10,cmap=plt.cm.RdBu_r,animated=True)

    stepx=10
    stepy=5
    
    Q = m.quiver(xx,yy,uproj,vproj,scale=700)
    # make quiver key.
    qk = plt.quiverkey(Q, 0.1, 0.1, 20, '20 m/s', labelpos='W')
    # draw coastlines, parallels, meridians.
    m.drawcoastlines(linewidth=1.5)
    m.drawparallels(parallels)
    m.drawmeridians(meridians)
    # add colorbar
    cb = m.colorbar(CS2,"right", size="5%", pad="2%")
    cb.set_label('m/s')
    # set plot title
    ax.set_title('Wind Speed at '+ datetime.datetime.strftime(date+datetime.timedelta(hours=tmax), '%a %b %d  %H:%M:%S %Z %Y'))
    

    if save :
    
    #compute direction
    
     theta=(180./np.pi)*np.arctan2(uproj,vproj)
        
    #write to shapefile
     loc=zip(xx.flatten(),yy.flatten(),vel.flatten(),theta.flatten())
    
     filename=datetime.datetime.strftime(date+datetime.timedelta(hours=tmax), '%Y%m%d%H')
    
     schema = { 'geometry': 'Point', 'properties': { 'vel': 'float', 'dir': 'float' } }
        
     with collection(SAVEPATH+'vector/'+filename+".shp", "w", "ESRI Shapefile", schema) as output:
        for x,y,v,th in loc:
            point = Point(x,y)
            output.write({
                'properties': {
                    'vel': v,
                    'dir': th
                },
                'geometry': mapping(point)
            })
    #write to geotiff
    
     TYPE=4     
     geo=(xx.min(),dd,0,yy.max(),0, -dd)  
     nodata=-9999.
     putmap(SAVEPATH+'/raster/wind.tif',velg,geo,TYPE,nodata)

     ds=datetime.datetime.strftime(date+datetime.timedelta(hours=tmax),'%a %b %d  %H:%M:%S %Z %Y')
     np.savetxt(SAVEPATH+'/txt/TIMESTAMP.txt',['{}   {}'.format(val,ds)],fmt='%s')

    fig1.savefig(SAVEPATH+'aegean_waves.png')


    plt.show()
    
    return tmax

In [ ]:
#test
tmax=wmap()

In [ ]:
#test
#tmax

In [ ]:
# Define range of file attributes
yy=[2015,2016]
yy=[w for w in map(str,yy )]
mm=np.arange(1,13)
mm=[w for w in map(str,mm )]
dd=np.arange(1,32)
dd=[w for w in map(str,dd )]
hh=[0,12]
hh=[w for w in map(str,hh )]



In [ ]:
tmax=interact_manual(wmap, yyyy=yy, mm=mm, dd=dd, hh=hh, save=widgets.Checkbox())

Now we plot the waves for this timestamp

In [ ]:
tmax.widget.result

get the values of the date stamp

In [ ]:
dd=tmax.widget.kwargs['dd']
mm=tmax.widget.kwargs['mm']
hh=tmax.widget.kwargs['hh']
yyyy=tmax.widget.kwargs['yyyy']

In [ ]:
# function for reading HNMS grib
def readgrib(INPUT):
    f = open(INPUT)

#    for t in range(3*time):
#        gid = grib_new_from_file(f,headers_only = True)

    for l in range(1):
        gid = grib_new_from_file(f)
        if gid is None: break

        name=grib_get(gid, 'shortName')

        ni = grib_get(gid, 'Ni')

        nj =  grib_get(gid, 'Nj')

        lat=grib_get_array(gid,'latitudes')
        lon=grib_get_array(gid,'longitudes')

        dat=grib_get_array(gid,'values')

        grib_release(gid)

    f.close()

    return dat.reshape(nj,ni),lat.reshape(nj,ni),lon.reshape(nj,ni)


In [ ]:
def map_waves(yyyy=2016,mm=2,dd=1,hh=0,time=1,save=False):
    hh=np.int(hh) # The variables are passed as string and needs to become integer to be used later
    yyyy=np.int(yyyy)
    mm=np.int(mm)
    dd=np.int(dd)
    time=np.int(time)
    # specify date to plot.
    date = datetime.datetime(yyyy,mm,dd,hh)

    # set PATH of the database.
    PATHbase="/mnt/Tsunamiweb/grib_HNMS_WAV/"  # Local mapping location for the above network drive
  # PATHbase="/mnt/ECMWF/grib/"  # Local mapping location for the above network drive
    PATH=PATHbase+"%04i/%02i/%02i/" % (yyyy,mm,dd)

    dpath=glob.glob(PATH+'*_{:04d}_{:02d}_{:02d}_{:02d}00_{:02d}h.grb'.format(yyyy,mm,dd,hh,time))

    try:
        data = pygrib.open(dpath[0])
    #    data,lats,lons = readgrib(dpath[0])
    # read lats,lons
        latlons=data[1].latlons()
    # reverse latitudes so they go from south to north.
        lats = np.flipud(latlons[0]) # make them increasing
        lons = latlons[1]
 #      lats=np.flipud(lats)
    # get significant height
        sh = data[1].data()[0]
 #      sh = np.ma.masked_equal(data,9999.)
        shi =np.flipud(sh) # make them increasing in lat
    except:
        print 'no available data in ', PATH
        return

    # make orthographic basemap.
    m = Basemap(projection='cyl',llcrnrlat=minlat,urcrnrlat=maxlat,\
             llcrnrlon=minlon,urcrnrlon=maxlon,resolution='h')
    # create figure, add axes
    fig2 = plt.figure(figsize=(8,10))
    ax = fig2.add_axes([0.1,0.1,0.8,0.8])
    # set desired contour levels.
    clevs = np.arange(960,1061,5)

    # compute native x,y coordinates of grid.
    x, y = m(lons, lats)
    # define parallels and meridians to draw.
    parallels = np.arange(-90.,90,10.)
    meridians = np.arange(0.,360.,10.)

    # plot SLP contours.
    CS1 = m.contour(x,y,shi,20,linewidths=0.5,colors='k',animated=True)
    CS2 = m.contourf(x,y,shi,20,cmap=plt.cm.RdBu_r,animated=True)
    # draw coastlines, parallels, meridians.
    m.drawcoastlines(linewidth=1.5)
    m.drawparallels(parallels)
    m.drawmeridians(meridians)
    # add colorbar
    cb = m.colorbar(CS2,"right", size="5%", pad="2%")
    cb.set_label('m')
    # set plot title
    ax.set_title('Significant height of combined wind waves and swell  \n'+ datetime.datetime.strftime(date+datetime.timedelta(hours=time), '%a %b %d  %H:%M:%S %Z %Y'))

    if save :
    # create a small matrix for the window

     latitudes = lats[:,0]
     longitudes = lons[0,:]

     wg=(latitudes > minlat) & (latitudes < maxlat) # 1-D mask latitudes
     zg=(longitudes > minlon) & (longitudes < maxlon) # 1-D mask longitudes BE CAREFUL OF & AND | DEPENDING

     i1,i2=np.argwhere(wg==True).min(),np.argwhere(wg==True).max()
     j1,j2=np.argwhere(zg==True).min(),np.argwhere(zg==True).max()

     ni,nj=np.argwhere(wg==True).size,np.argwhere(zg==True).size
     shgc=np.zeros((ni,nj))

     shgc=shi[i1:i2+1,j1:j2+1]
     shgc=np.flipud(shgc) # flip to save to geotif

    #write to geotiff

     TYPE=4

     geo=(longitudes[j1],data[1].jDirectionIncrementInDegrees,0,latitudes[i2],0, -data[1].iDirectionIncrementInDegrees)
#     geo=(lons.min(),data[1].iDirectionIncrementInDegrees,0,lats.max(),0, -data[1].jDirectionIncrementInDegrees)
     nodata=sh.fill_value
     putmap(SAVEPATH+'raster/sws.tif',shgc,geo,TYPE,nodata)


    fig2.savefig(SAVEPATH+'aegean_waves.png')
    plt.show()


In [ ]:
#test
#map_waves(time=)

In [ ]:
  map_waves( yyyy=yyyy,mm=mm,dd=dd,hh=hh,time=tmax.widget.result,save=False)